# Анализ таблиц JDWH - Прод vs Тест

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
pd.set_option('display.max_columns', 50)
pd.set_option('max_colwidth', 200)
#pd.set_option('display.max_rows', 500)

## Часть III. Индексы.

## 1. Загрузка и первичное преобразование данных

### Прод

In [3]:
prod = pd.read_csv('./data/indexes_prod_14122020.csv')

In [4]:
prod.head(3)

,schemaname,base_name,tablename,indexname,inds_count,ind,indexdef
0,dwh_bckp,amara_corr_overdues,amara_corr_overdues,pk_bckp_amara_corr_overdues,1,1,"CREATE UNIQUE INDEX pk_bckp_amara_corr_overdues ON dwh_bckp.amara_corr_overdues USING btree (id, insert_date)"
1,dwh_bckp,amara_rating_fitch,amara_rating_fitch,pk_bckp_amara_rating_fitch,1,1,"CREATE UNIQUE INDEX pk_bckp_amara_rating_fitch ON dwh_bckp.amara_rating_fitch USING btree (id, insert_date)"
2,dwh_bckp,amara_rating_fitch_forecast,amara_rating_fitch_forecast,pk_bckp_amara_rating_fitch_forecast,1,1,"CREATE UNIQUE INDEX pk_bckp_amara_rating_fitch_forecast ON dwh_bckp.amara_rating_fitch_forecast USING btree (id, insert_date)"


Добавим поле full_table_name

In [5]:
prod['full_table_name'] = prod['schemaname'] + '.' + prod['tablename']

Добавляем столбцы: признак уникальности (возможно это PK),тип индекска и его короткое определение

In [6]:
prod['uniq'] = prod.indexdef.apply(lambda x: 1 if 'CREATE UNIQUE INDEX' in x else 0)
prod['short_idx_def'] = prod.indexdef.apply(lambda x: x.split('USING')[1].strip())
prod['idx_type'] = prod.short_idx_def.apply(lambda x: x.split('(')[0].strip())
prod['ind_fields'] = prod.short_idx_def.apply(lambda x: x.split('(')[1].split(')')[0].strip())

In [7]:
prod.head(3)

,schemaname,base_name,tablename,indexname,inds_count,ind,indexdef,full_table_name,uniq,short_idx_def,idx_type,ind_fields
0,dwh_bckp,amara_corr_overdues,amara_corr_overdues,pk_bckp_amara_corr_overdues,1,1,"CREATE UNIQUE INDEX pk_bckp_amara_corr_overdues ON dwh_bckp.amara_corr_overdues USING btree (id, insert_date)",dwh_bckp.amara_corr_overdues,1,"btree (id, insert_date)",btree,"id, insert_date"
1,dwh_bckp,amara_rating_fitch,amara_rating_fitch,pk_bckp_amara_rating_fitch,1,1,"CREATE UNIQUE INDEX pk_bckp_amara_rating_fitch ON dwh_bckp.amara_rating_fitch USING btree (id, insert_date)",dwh_bckp.amara_rating_fitch,1,"btree (id, insert_date)",btree,"id, insert_date"
2,dwh_bckp,amara_rating_fitch_forecast,amara_rating_fitch_forecast,pk_bckp_amara_rating_fitch_forecast,1,1,"CREATE UNIQUE INDEX pk_bckp_amara_rating_fitch_forecast ON dwh_bckp.amara_rating_fitch_forecast USING btree (id, insert_date)",dwh_bckp.amara_rating_fitch_forecast,1,"btree (id, insert_date)",btree,"id, insert_date"


Убираем партиции

In [8]:
prod = prod[prod['schemaname'] + '.' + prod['tablename'] == prod['schemaname'] + '.' + prod['base_name']]
prod.sample(3)

,schemaname,base_name,tablename,indexname,inds_count,ind,indexdef,full_table_name,uniq,short_idx_def,idx_type,ind_fields
2231,dwh_dds,acc_doc_payment,acc_doc_payment,acc_doc_payment_docp_report_date_idx,1,1,CREATE INDEX acc_doc_payment_docp_report_date_idx ON dwh_dds.acc_doc_payment USING bitmap (docp_report_date),dwh_dds.acc_doc_payment,0,bitmap (docp_report_date),bitmap,docp_report_date
13,dwh_bckp,s01_led_acc_det_bkp,s01_led_acc_det_bkp,s01_led_acc_det_bkp_ch_id_idx,4,2,CREATE INDEX s01_led_acc_det_bkp_ch_id_idx ON dwh_bckp.s01_led_acc_det_bkp USING btree (ch_id),dwh_bckp.s01_led_acc_det_bkp,0,btree (ch_id),btree,ch_id
3871,dwh_dds,ref_pledge_link,ref_pledge_link,ref_pledge_link_tmp_v2_pledlnk_gid_idx,3,3,"CREATE INDEX ref_pledge_link_tmp_v2_pledlnk_gid_idx ON dwh_dds.ref_pledge_link USING btree (pledlnk_gid, ""pledlnk$start_date"", ""pledlnk$end_date"" DESC) WHERE ((""pledlnk$row_status"")::text = 'A'::t...",dwh_dds.ref_pledge_link,0,"btree (pledlnk_gid, ""pledlnk$start_date"", ""pledlnk$end_date"" DESC) WHERE ((""pledlnk$row_status"")::text = 'A'::text)",btree,"pledlnk_gid, ""pledlnk$start_date"", ""pledlnk$end_date"" DESC"


==============================================================================================================

### Тест

In [9]:
test = pd.read_csv('./data/indexes_test_14122020.csv')

In [10]:
test.head(3)

,schemaname,base_name,tablename,indexname,inds_count,ind,indexdef
0,dwh_bckp,amara_rating_fitch,amara_rating_fitch,pk_bckp_amara_rating_fitch,1,1,"CREATE UNIQUE INDEX pk_bckp_amara_rating_fitch ON dwh_bckp.amara_rating_fitch USING btree (id, insert_date)"
1,dwh_bckp,amara_rating_fitch_forecast,amara_rating_fitch_forecast,pk_bckp_amara_rating_fitch_forecast,1,1,"CREATE UNIQUE INDEX pk_bckp_amara_rating_fitch_forecast ON dwh_bckp.amara_rating_fitch_forecast USING btree (id, insert_date)"
2,dwh_bckp,amara_rating_fitch_short,amara_rating_fitch_short,pk_bckp_amara_rating_fitch_short,1,1,"CREATE UNIQUE INDEX pk_bckp_amara_rating_fitch_short ON dwh_bckp.amara_rating_fitch_short USING btree (id, insert_date)"


Добавим поле full_table_name

In [11]:
test['full_table_name'] = test['schemaname'] + '.' + test['tablename']

Добавляем столбцы: признак уникальности (возможно это PK),тип индекска и его короткое определение

In [12]:
test['uniq'] = test.indexdef.apply(lambda x: 1 if 'CREATE UNIQUE INDEX' in x else 0)
test['short_idx_def'] = test.indexdef.apply(lambda x: x.split('USING')[1].strip())
test['idx_type'] = test.short_idx_def.apply(lambda x: x.split('(')[0].strip())
test['ind_fields'] = test.short_idx_def.apply(lambda x: x.split('(')[1].split(')')[0].strip())

In [13]:
test.sample(3)

,schemaname,base_name,tablename,indexname,inds_count,ind,indexdef,full_table_name,uniq,short_idx_def,idx_type,ind_fields
7794,dwh_stage,s01_t_bal,s01_t_bal_1_prt_with_flzo_2_prt_236_2019_08_01,s01_t_bal_1_prt_with_flzo_2_prt_236_2019_08_01_bmx,2,2,CREATE INDEX s01_t_bal_1_prt_with_flzo_2_prt_236_2019_08_01_bmx ON dwh_stage.s01_t_bal_1_prt_with_flzo_2_prt_236_2019_08_01 USING bitmap (fromdate),dwh_stage.s01_t_bal_1_prt_with_flzo_2_prt_236_2019_08_01,0,bitmap (fromdate),bitmap,fromdate
5419,dwh_dm,dm_transaction,dm_transaction_1_prt_127,dm_transaction_1_prt_127_dm_trn_gid_idx,2,1,CREATE UNIQUE INDEX dm_transaction_1_prt_127_dm_trn_gid_idx ON dwh_dm.dm_transaction_1_prt_127 USING btree (dm_trn_gid),dwh_dm.dm_transaction_1_prt_127,1,btree (dm_trn_gid),btree,dm_trn_gid
9995,dwh_stage,s01_t_trndtl,s01_t_trndtl_1_prt_95_2020_10_09,s01_t_trndtl_1_prt_95_2020_10_09_doper_idx,4,3,CREATE INDEX s01_t_trndtl_1_prt_95_2020_10_09_doper_idx ON dwh_stage.s01_t_trndtl_1_prt_95_2020_10_09 USING bitmap (doper),dwh_stage.s01_t_trndtl_1_prt_95_2020_10_09,0,bitmap (doper),bitmap,doper


Убираем партиции

In [14]:
test = test[test['schemaname'] + '.' + test['tablename'] == test['schemaname'] + '.' + test['base_name']]
test.sample(3)

,schemaname,base_name,tablename,indexname,inds_count,ind,indexdef,full_table_name,uniq,short_idx_def,idx_type,ind_fields
4201,dwh_dds,ref_sc_identcode,ref_sc_identcode,ref_sc_identcode_tmp_v2_scidc_gid_idx,6,3,"CREATE INDEX ref_sc_identcode_tmp_v2_scidc_gid_idx ON dwh_dds.ref_sc_identcode USING btree (scidc_gid, ""scidc$start_date"", ""scidc$end_date"" DESC) WHERE ((""scidc$row_status"")::text = 'A'::text)",dwh_dds.ref_sc_identcode,0,"btree (scidc_gid, ""scidc$start_date"", ""scidc$end_date"" DESC) WHERE ((""scidc$row_status"")::text = 'A'::text)",btree,"scidc_gid, ""scidc$start_date"", ""scidc$end_date"" DESC"
6723,dwh_stage,s01_s_safevault,s01_s_safevault,pk_s01_s_safevault,1,1,CREATE UNIQUE INDEX pk_s01_s_safevault ON dwh_stage.s01_s_safevault USING btree (id),dwh_stage.s01_s_safevault,1,btree (id),btree,id
6612,dwh_stage,s01_g_accblnhst,s01_g_accblnhst,s01_g_accblnhst_v2_id_idx,3,3,"CREATE UNIQUE INDEX s01_g_accblnhst_v2_id_idx ON dwh_stage.s01_g_accblnhst USING btree (id, fromdate DESC)",dwh_stage.s01_g_accblnhst,1,"btree (id, fromdate DESC)",btree,"id, fromdate DESC"


In [15]:
prod.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 581 entries, 0 to 7959
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   schemaname       581 non-null    object
 1   base_name        581 non-null    object
 2   tablename        581 non-null    object
 3   indexname        581 non-null    object
 4   inds_count       581 non-null    int64 
 5   ind              581 non-null    int64 
 6   indexdef         581 non-null    object
 7   full_table_name  581 non-null    object
 8   uniq             581 non-null    int64 
 9   short_idx_def    581 non-null    object
 10  idx_type         581 non-null    object
 11  ind_fields       581 non-null    object
dtypes: int64(3), object(9)
memory usage: 59.0+ KB


In [16]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 594 entries, 0 to 11540
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   schemaname       594 non-null    object
 1   base_name        594 non-null    object
 2   tablename        594 non-null    object
 3   indexname        594 non-null    object
 4   inds_count       594 non-null    int64 
 5   ind              594 non-null    int64 
 6   indexdef         594 non-null    object
 7   full_table_name  594 non-null    object
 8   uniq             594 non-null    int64 
 9   short_idx_def    594 non-null    object
 10  idx_type         594 non-null    object
 11  ind_fields       594 non-null    object
dtypes: int64(3), object(9)
memory usage: 60.3+ KB


### Сверяем только те таблицы, что есть и на Прод и на Тест

Прод

In [17]:
#Замечание: проверки такого типа позволяют выявить нестандартные определения индексов
#Но сейчас не понадобилось - не используем
#prod[prod.ind_fields.str.strip()=='']
#prod[prod.indexdef.str.strip().str.contains('::')]

Удалим лишние столбцы

In [18]:
#Удаляем лишние столбцы
prod.drop(['base_name', 'short_idx_def'], inplace=True, axis = 1)#, 'indexdef'
prod.sample(3)

,schemaname,tablename,indexname,inds_count,ind,indexdef,full_table_name,uniq,idx_type,ind_fields
3803,dwh_dds,ref_deal_forex,ref_deal_forex_dlfx_prnt_deal_dprt_gid_idx,16,11,CREATE INDEX ref_deal_forex_dlfx_prnt_deal_dprt_gid_idx ON dwh_dds.ref_deal_forex USING btree (dlfx_prnt_deal_dprt_gid),dwh_dds.ref_deal_forex,0,btree,dlfx_prnt_deal_dprt_gid
7958,dwh_stage,s03_lfa1,s03_lfa1_lifnr_idx,1,1,CREATE INDEX s03_lfa1_lifnr_idx ON dwh_stage.s03_lfa1 USING btree (lifnr),dwh_stage.s03_lfa1,0,btree,lifnr
3842,dwh_dds,ref_deal_schedule,ref_deal_schedule_tmp_v2_sched_gid_idx,2,2,"CREATE INDEX ref_deal_schedule_tmp_v2_sched_gid_idx ON dwh_dds.ref_deal_schedule USING btree (sched_gid, ""sched$start_date"", ""sched$end_date"" DESC) WHERE ((""sched$row_status"")::text = 'A'::text)",dwh_dds.ref_deal_schedule,0,btree,"sched_gid, ""sched$start_date"", ""sched$end_date"" DESC"


Test

Удалим лишние столбцы

In [19]:
#Удаляем лишние столбцы
test.drop(['base_name', 'short_idx_def'], inplace=True, axis = 1)#, 'indexdef'
test.sample(3)

,schemaname,tablename,indexname,inds_count,ind,indexdef,full_table_name,uniq,idx_type,ind_fields
6646,dwh_stage,s01_g_dbljrndtl,g_dbljrndtl,1,1,"CREATE UNIQUE INDEX g_dbljrndtl ON dwh_stage.s01_g_dbljrndtl USING btree (jrn_id, code, nord)",dwh_stage.s01_g_dbljrndtl,1,btree,"jrn_id, code, nord"
4085,dwh_dds,ref_auth_person,pk_ref_auth_person,1,1,"CREATE INDEX pk_ref_auth_person ON dwh_dds.ref_auth_person USING btree (pst_gid, ""pst$end_date"" DESC, ""pst$row_status"")",dwh_dds.ref_auth_person,0,btree,"pst_gid, ""pst$end_date"" DESC, ""pst$row_status"""
8215,dwh_stage,s01_t_bop_stat_std,s01_t_bop_stat_std_pk,1,1,"CREATE UNIQUE INDEX s01_t_bop_stat_std_pk ON dwh_stage.s01_t_bop_stat_std USING btree (id, nord)",dwh_stage.s01_t_bop_stat_std,1,btree,"id, nord"


### Выравниваем Прод и Тест

In [20]:
prod2 = prod[['full_table_name']]
prod2.head(2)

,full_table_name
0,dwh_bckp.amara_corr_overdues
1,dwh_bckp.amara_rating_fitch


In [21]:
test2 = test[['full_table_name']]
test2.head(2)

,full_table_name
0,dwh_bckp.amara_rating_fitch
1,dwh_bckp.amara_rating_fitch_forecast


In [22]:
test2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 594 entries, 0 to 11540
Data columns (total 1 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   full_table_name  594 non-null    object
dtypes: object(1)
memory usage: 9.3+ KB


Готовим список нужных имен таблиц (которые есть и на Прод и на Тест)

In [23]:
prod2_test2_merge = prod2.merge(test2, on='full_table_name', how='inner')
prod2_test2_merge.head(2)

,full_table_name
0,dwh_bckp.amara_rating_fitch
1,dwh_bckp.amara_rating_fitch_forecast


In [24]:
#Эти таблицы есть и на Прод и на Тест. Остальные будем удалять.
not_to_delete = list(prod2_test2_merge.full_table_name.value_counts().index)

In [25]:
not_to_delete[:10] #Первые 10 элементов

['dwh_dds.ref_sc_issue',
 'dwh_dds.ref_deal_forex',
 'dwh_dds.ref_sc_lot',
 'dwh_dds.ref_sc_ticket',
 'dwh_dds.ref_account',
 'dwh_dds.ref_deal_scissue',
 'dwh_dds.acc_sc_quote',
 'dwh_dds.ref_gen_agreement',
 'dwh_dds.ref_sc_rating',
 'dwh_dds.ref_sc_identcode']

Теперь удаляем строки в Прод и Тест, не входящие в список по полю full_table_name

Прод

In [26]:
indices = prod[~(prod['full_table_name'].isin(not_to_delete))].index
prod.drop(indices, inplace=True)

Тест

In [27]:
indices = test[~(test['full_table_name'].isin(not_to_delete))].index
test.drop(indices, inplace=True)

### Теперь наборы данных Прод и Тест содержат одинаковые таблицы и готовы к сравнению инднксов

In [28]:
len(prod)

573

In [29]:
len(test)

566

Сравнение таблиц по количеству индексов:

In [30]:
tabs_different_number_of_indices = set()

In [31]:
def check_indexes_quantity(row):
    '''Сравнивает количество индексов для таблиц на Прод и Тест и
       если оно различно, добавляет таблицу во множество tabs_different_number_of_indices '''
    if row['inds_count'] != test[test['full_table_name'] == row['full_table_name']].inds_count.max():
        tabs_different_number_of_indices.add(row['full_table_name'])

In [32]:
prod.apply(lambda r: check_indexes_quantity(r), axis=1)

1       None
2       None
3       None
4       None
5       None
        ... 
7955    None
7956    None
7957    None
7958    None
7959    None
Length: 573, dtype: object

### Вот список таблиц, индексы которых будем сверять вручную:

In [33]:
list(tabs_different_number_of_indices)

['dwh_dds.acc_doc_payment',
 'dwh_stage.s01_t_deashdhst',
 'dwh_dds.ref_deal_scissue',
 'dwh_dds.ref_sc_rating',
 'dwh_stage.s01_t_operjrn',
 'dwh_dm.dm_transaction',
 'dwh_stage.s01_s_ordcash_add',
 'dwh_stage.s01_g_accblnhst',
 'dwh_dds.ref_sc_issue',
 'dwh_dds.ref_anl_account',
 'dwh_stage.s01_ansignval',
 'dwh_dds.ref_interbank_deals',
 'dwh_dds.ref_sc_ticket']

Теперь удалим строки с этими таблицами из Прод и Тест. Остальное можно попробовать сверить в автоматическом режиме

In [34]:
#Общий список таблиц для сверки вручную
tabs_to_manually_comparison = list(tabs_different_number_of_indices)
tabs_to_manually_comparison

['dwh_dds.acc_doc_payment',
 'dwh_stage.s01_t_deashdhst',
 'dwh_dds.ref_deal_scissue',
 'dwh_dds.ref_sc_rating',
 'dwh_stage.s01_t_operjrn',
 'dwh_dm.dm_transaction',
 'dwh_stage.s01_s_ordcash_add',
 'dwh_stage.s01_g_accblnhst',
 'dwh_dds.ref_sc_issue',
 'dwh_dds.ref_anl_account',
 'dwh_stage.s01_ansignval',
 'dwh_dds.ref_interbank_deals',
 'dwh_dds.ref_sc_ticket']

In [35]:
#Удаляем строки с этими таблицами:
indices = prod[prod['full_table_name'].isin(tabs_to_manually_comparison)].index
prod.drop(indices, inplace=True)

indices = test[test['full_table_name'].isin(tabs_to_manually_comparison)].index
test.drop(indices, inplace=True)

In [36]:
len(prod), len(test)

(502, 502)

**Видим, что на Прод и Тест осталось одинаковое количество строк!**

Сначала попробкем сверить уникальные индексы (primary keys)

Определим таблицы с более чем одним уникальным индексом на Проде и Тесте отдельно:

In [37]:
prod[prod.uniq==1].full_table_name.value_counts()

dwh_stage.s01_t_acc             2
dwh_stage.s01_t_pcndsc_std      2
dwh_stage.s01_g_address         1
dwh_stage.s01_c_clndclc         1
dwh_stage.s01_l_aprrecalc       1
                               ..
dwh_stage.s01_t_deaaccrul       1
dwh_stage.s01_t_bop_dscr_std    1
dwh_stage.s01_t_ordcms          1
dwh_stage.s01_m_ordpaydtl       1
dwh_bckp.amara_rating_scale     1
Name: full_table_name, Length: 298, dtype: int64

In [38]:
test[test.uniq==1].full_table_name.value_counts()

dwh_stage.s01_t_pcndsc_std      2
dwh_stage.s01_t_acc             2
dwh_stage.s01_g_address         1
dwh_stage.s01_t_oprchr_std      1
dwh_stage.s01_c_clndclc         1
                               ..
dwh_stage.s01_t_deaaccrul       1
dwh_stage.s01_t_bop_dscr_std    1
dwh_stage.s01_t_ordcms          1
dwh_stage.s01_m_ordpaydtl       1
dwh_bckp.amara_rating_scale     1
Name: full_table_name, Length: 297, dtype: int64

**На Прод и Тест таблицы с более чем одним уникальным индексом одни и те же, НО!, группировка по именам таблиц показывает,
что на Тесте для какой-то таблицы нет уникального индекса при его налияии на Проде. Разные количества строк: 297 на Прод и 296 на Тест**

In [39]:
tabs_diffrent_count_of_unique_indexes = ['dwh_stage.s01_t_acc', 'dwh_stage.s01_t_pcndsc_std']

Добавляем этот список к списку для сверки вручную:

In [40]:
tabs_to_manually_comparison += tabs_diffrent_count_of_unique_indexes
tabs_to_manually_comparison

['dwh_dds.acc_doc_payment',
 'dwh_stage.s01_t_deashdhst',
 'dwh_dds.ref_deal_scissue',
 'dwh_dds.ref_sc_rating',
 'dwh_stage.s01_t_operjrn',
 'dwh_dm.dm_transaction',
 'dwh_stage.s01_s_ordcash_add',
 'dwh_stage.s01_g_accblnhst',
 'dwh_dds.ref_sc_issue',
 'dwh_dds.ref_anl_account',
 'dwh_stage.s01_ansignval',
 'dwh_dds.ref_interbank_deals',
 'dwh_dds.ref_sc_ticket',
 'dwh_stage.s01_t_acc',
 'dwh_stage.s01_t_pcndsc_std']

In [41]:
#Имеем 17 таблиц для сверки индексов вручную:
len(tabs_to_manually_comparison)

15

In [42]:
#Удаляем строки с этими таблицами:
indices = prod[prod['full_table_name'].isin(tabs_diffrent_count_of_unique_indexes)].index
prod.drop(indices, inplace=True)

indices = test[test['full_table_name'].isin(tabs_diffrent_count_of_unique_indexes)].index
test.drop(indices, inplace=True)

In [43]:
len(prod), len(test)

(498, 498)

In [44]:
tabs_diffrent_unique_indexes = set()

In [45]:
def extract_nonunique_indexes_test(row):
    '''Определяет таблицы с неуникальными индексами на Тесте при их уникальности на Проде'''
    
    ind_type_on_test = test[(test.full_table_name==row.full_table_name) & (test.uniq==1)].idx_type
    if len(ind_type_on_test) > 1:
        raise ValueError('Table ' + row.full_table_name + ' has more then one unuque index')
    
    if len(ind_type_on_test) < 1:
        print('На Тесте у таблицы ' + row.full_table_name + ' нет уникальных индексов (на Проде есть)') 
        tabs_diffrent_unique_indexes.add(row['full_table_name'])

In [46]:
prod[prod.uniq==1].apply(lambda r: extract_nonunique_indexes_test(r), axis=1)

На Тесте у таблицы dwh_stage.s01_p_csrjrn нет уникальных индексов (на Проде есть)


1       None
2       None
3       None
4       None
5       None
        ... 
7947    None
7948    None
7950    None
7951    None
7959    None
Length: 296, dtype: object

In [47]:
def extract_nonunique_indexes_prod(row):
    '''Определяет таблицы с неуникальными индексами на Проде при их уникальности на Тесте'''
    
    ind_type_on_prod = prod[(prod.full_table_name==row.full_table_name) & (prod.uniq==1)].idx_type
    if len(ind_type_on_prod) > 1:
        raise ValueError('Table ' + row.full_table_name + ' has more then one unuque index')
    
    if len(ind_type_on_prod) < 1:
        print('На Проде у таблицы ' + row.full_table_name + ' нет уникальных индексов (на Тесте есть)') 
        tabs_diffrent_unique_indexes.add(row['full_table_name'])

In [48]:
test[test.uniq==1].apply(lambda r: extract_nonunique_indexes_prod(r), axis=1)

0        None
1        None
2        None
3        None
4        None
         ... 
11527    None
11528    None
11530    None
11531    None
11540    None
Length: 295, dtype: object

Таблицы с различием по уникальности индексов на Прод и Тест:

In [49]:
list(tabs_diffrent_unique_indexes)

['dwh_stage.s01_p_csrjrn']

Удалим эту таблицу из рассмотрения на Прод и Тест:

In [50]:
indices = prod[prod['full_table_name'].isin(list(tabs_diffrent_unique_indexes))].index
prod.drop(indices, inplace=True)

indices = test[test['full_table_name'].isin(list(tabs_diffrent_unique_indexes))].index
test.drop(indices, inplace=True)

In [51]:
len(prod), len(test)

(497, 497)

Теперь только можно попробовать сверить уникальные индексы по типам!

In [52]:
#Оставшиеся для анализа (количество) уникальные индексы на Прод и Тест
len(prod[prod.uniq==1]), len(test[test.uniq==1])

(295, 295)

In [53]:
def compare_unique_indexes_types(row):
    '''Сверяет уникальные индексы по типу'''
    #Сначала сверяем по типу:
    ind_type_on_test = test[(test.full_table_name==row.full_table_name) & (test.uniq==1)].idx_type
    
    if len(ind_type_on_test) > 1:
        raise ValueError('Table ' + row.full_table_name + ' has more then one unuque index')
    
    if len(ind_type_on_test) < 1:
        raise ValueError('Table ' + row.full_table_name + ' has less then one unuque index')
    
    if ind_type_on_test.iloc[0] != row.idx_type:
        tabs_diffrent_unique_indexes.add(row['full_table_name'])

In [54]:
prod[prod.uniq==1].apply(lambda r: compare_unique_indexes_types(r), axis=1)

1       None
2       None
3       None
4       None
5       None
        ... 
7947    None
7948    None
7950    None
7951    None
7959    None
Length: 295, dtype: object

In [55]:
tabs_diffrent_unique_indexes

{'dwh_stage.s01_p_csrjrn'}

Никакие таблицы в список не добавились. Значит типы уникальных индексов на Прод и Тест совпадают

Теперь пробуем сверять уникальные индексы по составу полей

In [56]:
def compare_unique_indexes_columns(row):
    '''Сверяет уникальные индексы по полям'''
    #Сначала сверяем по типу:
    ind_columns_on_test = test[(test.full_table_name==row.full_table_name) & (test.uniq==1)].ind_fields
    
    if len(ind_columns_on_test) > 1:
        raise ValueError('Table ' + row.full_table_name + ' has more then one unuque index')
    
    if len(ind_columns_on_test) < 1:
        raise ValueError('Table ' + row.full_table_name + ' has less then one unuque index')
    
    if ind_columns_on_test.iloc[0] != row.ind_fields:
        tabs_diffrent_unique_indexes.add(row['full_table_name'])

In [57]:
prod[prod.uniq==1].apply(lambda r: compare_unique_indexes_columns(r), axis=1)

1       None
2       None
3       None
4       None
5       None
        ... 
7947    None
7948    None
7950    None
7951    None
7959    None
Length: 295, dtype: object

In [58]:
tabs_diffrent_unique_indexes

{'dwh_stage.s01_p_csrjrn'}

Отлично! Поля в определениях уникальных индексов на Прод и Тест одни и те же!

Добавляем к общему списку таблиц для проверки вручную:

In [59]:
tabs_to_manually_comparison += list(tabs_diffrent_unique_indexes)

### Теперь уберем строки с уникальными инднксами и рассмотрим только обычные индексы

In [60]:
indices = prod[prod['uniq']==1].index
prod.drop(indices, inplace=True)

indices = test[test['uniq']==1].index
test.drop(indices, inplace=True)

In [61]:
len(prod), len(test)

(202, 202)

In [62]:
prod.head()

,schemaname,tablename,indexname,inds_count,ind,indexdef,full_table_name,uniq,idx_type,ind_fields
12,dwh_bckp,s01_led_acc_det_bkp,s01_led_acc_det_bkp_acc_id_idx,4,1,CREATE INDEX s01_led_acc_det_bkp_acc_id_idx ON dwh_bckp.s01_led_acc_det_bkp USING btree (acc_id),dwh_bckp.s01_led_acc_det_bkp,0,btree,acc_id
13,dwh_bckp,s01_led_acc_det_bkp,s01_led_acc_det_bkp_ch_id_idx,4,2,CREATE INDEX s01_led_acc_det_bkp_ch_id_idx ON dwh_bckp.s01_led_acc_det_bkp USING btree (ch_id),dwh_bckp.s01_led_acc_det_bkp,0,btree,ch_id
14,dwh_bckp,s01_led_acc_det_bkp,s01_led_acc_det_bkp_pk2_idx,4,3,CREATE INDEX s01_led_acc_det_bkp_pk2_idx ON dwh_bckp.s01_led_acc_det_bkp USING btree (((pk2)::integer)),dwh_bckp.s01_led_acc_det_bkp,0,btree,
15,dwh_bckp,s01_led_acc_det_bkp,s01_led_acc_det_bkp_sgn_id_bmx,4,4,CREATE INDEX s01_led_acc_det_bkp_sgn_id_bmx ON dwh_bckp.s01_led_acc_det_bkp USING bitmap (sgn_id),dwh_bckp.s01_led_acc_det_bkp,0,bitmap,sgn_id
1484,dwh_bckp,s01_t_dea_bkp,s01_t_dea_bkp_cli_id_idx,2,1,CREATE INDEX s01_t_dea_bkp_cli_id_idx ON dwh_bckp.s01_t_dea_bkp USING btree (cli_id),dwh_bckp.s01_t_dea_bkp,0,btree,cli_id


In [63]:
test.head()

,schemaname,tablename,indexname,inds_count,ind,indexdef,full_table_name,uniq,idx_type,ind_fields
14,dwh_bckp,s01_led_acc_det_bkp,s01_led_acc_det_bkp_acc_id_idx,4,1,CREATE INDEX s01_led_acc_det_bkp_acc_id_idx ON dwh_bckp.s01_led_acc_det_bkp USING btree (acc_id),dwh_bckp.s01_led_acc_det_bkp,0,btree,acc_id
15,dwh_bckp,s01_led_acc_det_bkp,s01_led_acc_det_bkp_ch_id_idx,4,2,CREATE INDEX s01_led_acc_det_bkp_ch_id_idx ON dwh_bckp.s01_led_acc_det_bkp USING btree (ch_id),dwh_bckp.s01_led_acc_det_bkp,0,btree,ch_id
16,dwh_bckp,s01_led_acc_det_bkp,s01_led_acc_det_bkp_pk2_idx,4,3,CREATE INDEX s01_led_acc_det_bkp_pk2_idx ON dwh_bckp.s01_led_acc_det_bkp USING btree (((pk2)::integer)),dwh_bckp.s01_led_acc_det_bkp,0,btree,
17,dwh_bckp,s01_led_acc_det_bkp,s01_led_acc_det_bkp_sgn_id_bmx,4,4,CREATE INDEX s01_led_acc_det_bkp_sgn_id_bmx ON dwh_bckp.s01_led_acc_det_bkp USING bitmap (sgn_id),dwh_bckp.s01_led_acc_det_bkp,0,bitmap,sgn_id
1486,dwh_bckp,s01_t_dea_bkp,s01_t_dea_bkp_cli_id_idx,2,1,CREATE INDEX s01_t_dea_bkp_cli_id_idx ON dwh_bckp.s01_t_dea_bkp USING btree (cli_id),dwh_bckp.s01_t_dea_bkp,0,btree,cli_id


Получим список таблиц для сверки неуникальных индексов (список содержит те же таблицы на Тест и Прод):

In [64]:
tables_to_check_nununique_indexes = list(prod.full_table_name.value_counts().index)
tables_to_check_nununique_indexes

['dwh_dds.ref_deal_forex',
 'dwh_dds.ref_sc_lot',
 'dwh_dds.ref_account',
 'dwh_dds.ref_gen_agreement',
 'dwh_dds.acc_sc_quote',
 'dwh_dds.ref_pledge_object',
 'dwh_dds.ref_bank_corr_relation',
 'dwh_dds.ref_sc_identcode',
 'dwh_dds.ref_deal_letter_credit',
 'dwh_bckp.s01_led_acc_det_bkp',
 'dwh_dds.ref_deal_guarantees',
 'dwh_stage.s01_t_trndtl',
 'dwh_dds.ref_deal_pledge',
 'dwh_dds.ref_deal_current_account',
 'dwh_dds.ref_pledge_obj_link',
 'dwh_dds.acc_transaction',
 'dwh_stage.s01_s_ordcash',
 'dwh_dds.ref_client_corp',
 'dwh_dds.acc_sc_deal_lot_link',
 'dwh_stage.s01_g_accbln',
 'dwh_stage.s01_ledacc_det',
 'dwh_dds.ref_payment_card',
 'dwh_dds.ref_pledge_link',
 'dwh_stage.s01_t_procinh',
 'dwh_stage.s01_t_procmem',
 'dwh_dds.acc_anl_account_balance',
 'dwh_stage.s03_bsak',
 'dwh_dds.ref_account_overdue',
 'dwh_stage.s01_l_mrtordobj',
 'dwh_dds.ref_client_priv',
 'dwh_dds.ref_safe',
 'dwh_dm.dm_z10',
 'dwh_stage.s01_l_mrtorddea',
 'dwh_stage.s01_g_identdocdsc_std',
 'dwh_stage.s

In [65]:
print('Всего уникальных таблиц:', len(tables_to_check_nununique_indexes))

Всего уникальных таблиц: 90


Создаем список таблиц с несовпадающими НЕУНИКАЛЬНЫМИ индексами

In [66]:
tabs_diffrent_nonunique_indexes = set()

Пробуем сверить просто по определению индекса

In [67]:
for tab in tables_to_check_nununique_indexes:
    sub_set_prod = prod[prod.full_table_name==tab]
    #display(sub_set_prod)
    
    sub_set_test = test[test.full_table_name==tab]
    #display(sub_set_test)
    
    if set(sub_set_prod.indexdef.value_counts().index) != set(sub_set_test.indexdef.value_counts().index):
        tabs_diffrent_nonunique_indexes.add(tab)
        
list(tabs_diffrent_nonunique_indexes)

['dwh_dds.ref_account',
 'dwh_dm.dm_z10',
 'dwh_stage.s01_t_trndtl_tmp',
 'dwh_stage.s01_p_ord']

Добавим к списку всех таблиц для проверки вручную:

In [68]:
tabs_to_manually_comparison += list(tabs_diffrent_nonunique_indexes)

## Итого, нужно проверить следующие списки таблиц:

1. Таблицы с различным количеством индексов:

In [69]:
list(tabs_different_number_of_indices)

['dwh_dds.acc_doc_payment',
 'dwh_stage.s01_t_deashdhst',
 'dwh_dds.ref_deal_scissue',
 'dwh_dds.ref_sc_rating',
 'dwh_stage.s01_t_operjrn',
 'dwh_dm.dm_transaction',
 'dwh_stage.s01_s_ordcash_add',
 'dwh_stage.s01_g_accblnhst',
 'dwh_dds.ref_sc_issue',
 'dwh_dds.ref_anl_account',
 'dwh_stage.s01_ansignval',
 'dwh_dds.ref_interbank_deals',
 'dwh_dds.ref_sc_ticket']

2. Таблицы с более чем одним уникальным индексом:

In [70]:
tabs_diffrent_count_of_unique_indexes

['dwh_stage.s01_t_acc', 'dwh_stage.s01_t_pcndsc_std']

3. Таблицы, где уникальность индексов не совпадает на Прод и Тест

In [71]:
tabs_diffrent_unique_indexes

{'dwh_stage.s01_p_csrjrn'}

4. Таблицы с различным определением неуникальных индексов

In [72]:
list(tabs_diffrent_nonunique_indexes)

['dwh_dds.ref_account',
 'dwh_dm.dm_z10',
 'dwh_stage.s01_t_trndtl_tmp',
 'dwh_stage.s01_p_ord']

**5. Итого, все аномальные таблицы**

In [73]:
tabs_to_manually_comparison

['dwh_dds.acc_doc_payment',
 'dwh_stage.s01_t_deashdhst',
 'dwh_dds.ref_deal_scissue',
 'dwh_dds.ref_sc_rating',
 'dwh_stage.s01_t_operjrn',
 'dwh_dm.dm_transaction',
 'dwh_stage.s01_s_ordcash_add',
 'dwh_stage.s01_g_accblnhst',
 'dwh_dds.ref_sc_issue',
 'dwh_dds.ref_anl_account',
 'dwh_stage.s01_ansignval',
 'dwh_dds.ref_interbank_deals',
 'dwh_dds.ref_sc_ticket',
 'dwh_stage.s01_t_acc',
 'dwh_stage.s01_t_pcndsc_std',
 'dwh_stage.s01_p_csrjrn',
 'dwh_dds.ref_account',
 'dwh_dm.dm_z10',
 'dwh_stage.s01_t_trndtl_tmp',
 'dwh_stage.s01_p_ord']

In [74]:
len(tabs_to_manually_comparison)

20